In [1]:
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
saver = tf.train.import_meta_graph("save/nn.ckpt.meta")

In [3]:
# from keras.applications import VGG16

# model = VGG16(weights='imagenet',
#               include_top=False)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



Using TensorFlow backend.


In [4]:
g = tf.get_default_graph()
for op in g.get_operations():
    print(op.name)

input_1
block1_conv1/random_uniform/shape
block1_conv1/random_uniform/min
block1_conv1/random_uniform/max
block1_conv1/random_uniform/RandomUniform
block1_conv1/random_uniform/sub
block1_conv1/random_uniform/mul
block1_conv1/random_uniform
block1_conv1/kernel
block1_conv1/kernel/IsInitialized/VarIsInitializedOp
block1_conv1/kernel/Assign
block1_conv1/kernel/Read/ReadVariableOp
block1_conv1/Const
block1_conv1/bias
block1_conv1/bias/IsInitialized/VarIsInitializedOp
block1_conv1/bias/Assign
block1_conv1/bias/Read/ReadVariableOp
block1_conv1/convolution/ReadVariableOp
block1_conv1/convolution
block1_conv1/BiasAdd/ReadVariableOp
block1_conv1/BiasAdd
block1_conv1/Relu
block1_conv2/random_uniform/shape
block1_conv2/random_uniform/min
block1_conv2/random_uniform/max
block1_conv2/random_uniform/RandomUniform
block1_conv2/random_uniform/sub
block1_conv2/random_uniform/mul
block1_conv2/random_uniform
block1_conv2/kernel
block1_conv2/kernel/IsInitialized/VarIsInitializedOp
block1_conv2/kernel/Assi

In [5]:
layer_name = 'block3_conv1/Relu:0'
filter_index = 0

g = tf.get_default_graph()
x = g.get_tensor_by_name('input_1:0')
layer_out = g.get_tensor_by_name('block3_conv1/Relu:0')
layer_filter_out = layer_out[:, :, :, filter_index]
loss = -tf.reduce_mean(layer_filter_out)
init = g.get_operation_by_name('init') 

grads = tf.gradients(loss, [x])[0] # auto gradient
grads /= (tf.sqrt(tf.reduce_mean(tf.square(grads))) + 1e-5)

In [6]:
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x = x.astype(np.float32)
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [7]:
with tf.Session() as sess:
    saver.restore(sess, 'save/nn.ckpt')
    #init.run()
#     g = tf.get_default_graph()
#     sess.run(g.get_operation_by_name('AssignVariableOp'))
#     for i in range(1,26):
#         sess.run(g.get_operation_by_name('AssignVariableOp_{}'.format(i)))
        
    
    img = np.random.random((1, 150, 150, 3)) * 20 + 128.
    
    # Run gradient ascent for 40 steps
    lr = 1.  # this is the magnitude of each gradient update
    loss_run = []
    for i in range(40):
        feed_dict = {x: img} 
        # Compute the loss value and gradient value
        loss_value, grads_value = sess.run([loss,grads], feed_dict=feed_dict)
        # Here we adjust the input image in the direction that maximizes the loss
        img -= grads_value * lr
        loss_run.append(loss_value)
    img = deprocess_image(img)
    plt.imshow(img[0])
    plt.show()
    
    plt.plot(loss_run)
    plt.show()

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,?,?,?]
	 [[node Placeholder (defined at /Users/sungchullee/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'Placeholder':
  File "/Users/sungchullee/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/sungchullee/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/sungchullee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/sungchullee/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/Users/sungchullee/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/Users/sungchullee/anaconda3/lib/python3.6/

In [ ]:
x = g.get_tensor_by_name('input_1:0')
block1_conv1 = g.get_tensor_by_name('block1_conv1/Relu:0')
block1_conv2 = g.get_tensor_by_name('block1_conv2/Relu:0')
block2_conv1 = g.get_tensor_by_name('block2_conv1/Relu:0')
block2_conv2 = g.get_tensor_by_name('block2_conv2/Relu:0')
block3_conv1 = g.get_tensor_by_name('block3_conv1/Relu:0')
block3_conv2 = g.get_tensor_by_name('block3_conv2/Relu:0')
block3_conv3 = g.get_tensor_by_name('block3_conv3/Relu:0')
tensor_list = [block1_conv1, block1_conv2,
               block2_conv1, block2_conv2,
               block3_conv1, block3_conv2, block3_conv3]
tensor_name_list = ['block1_conv1/Relu:0', 'block1_conv2/Relu:0',
                    'block2_conv1/Relu:0', 'block2_conv2/Relu:0',
                    'block3_conv1/Relu:0', 'block3_conv2/Relu:0', 'block3_conv3/Relu:0']

In [ ]:
def generate_pattern(layer_name, filter_index, size=150):
    g = tf.get_default_graph()
    x = g.get_tensor_by_name('input_1:0')
    layer_out = g.get_tensor_by_name(layer_name)
    layer_filter_out = layer_out[:, :, :, filter_index]
    loss = tf.reduce_mean(layer_filter_out)

    grads = tf.gradients(loss, x)[0] # auto gradient
    grads /= (tf.sqrt(tf.reduce_mean(tf.square(grads))) + 1e-5)
    
    with tf.Session() as sess:
        #saver.restore(sess, 'save/nn.ckpt')
    
        img = np.random.random((1, 150, 150, 3)) * 20 + 128.
        lr = 1.  
        for i in range(40):
            feed_dict = {x: img} 
            loss_value, grads_value = sess.run([loss,grads], feed_dict=feed_dict)
            img += grads_value * lr
        img = deprocess_image(img)
    return img[0]

In [ ]:
img = generate_pattern(layer_name='block1_conv1/Relu:0',
                       filter_index=0)
plt.imshow(img)
plt.show()

In [ ]:
def generate_patterns(layer_name):
    g = tf.get_default_graph()
    layer_out = g.get_tensor_by_name(layer_name)
    _, _, _, n_filters = layer_out.shape
    img_out = []
    for i in range(n_filters):
        img = generate_pattern(layer_name, i)
        img_out.append(img)
    return np.array(img_out).astype(np.float32)

In [ ]:
imgs = generate_patterns(layer_name='block1_conv1/Relu:0')
print(imgs.shape)

In [ ]:
def display_patterns(layer_name): 
    
    imgs = generate_patterns(layer_name) 
    
    images_per_row = 16
    n_imgs = imgs.shape[0] 
    size = imgs.shape[1]

    n_cols = n_imgs // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size, 3))

    # We'll tile each filter into this big horizontal grid
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = imgs[col * images_per_row + row]
            # Post-process the feature to make it visually palatable
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image

    # Display the grid
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    
    plt.show()

In [ ]:
display_patterns(layer_name='block1_conv1/Relu:0')

In [ ]:
x = g.get_tensor_by_name('input_1:0')
block1_conv1 = g.get_tensor_by_name('block1_conv1/Relu:0')
block1_conv2 = g.get_tensor_by_name('block1_conv2/Relu:0')
block2_conv1 = g.get_tensor_by_name('block2_conv1/Relu:0')
block2_conv2 = g.get_tensor_by_name('block2_conv2/Relu:0')
block3_conv1 = g.get_tensor_by_name('block3_conv1/Relu:0')
block3_conv2 = g.get_tensor_by_name('block3_conv2/Relu:0')
block3_conv3 = g.get_tensor_by_name('block3_conv3/Relu:0')
tensor_list = [block1_conv1, block1_conv2,
               block2_conv1, block2_conv2,
               block3_conv1, block3_conv2, block3_conv3]
tensor_name_list = ['block1_conv1/Relu:0', 'block1_conv2/Relu:0',
                    'block2_conv1/Relu:0', 'block2_conv2/Relu:0',
                    'block3_conv1/Relu:0', 'block3_conv2/Relu:0', 'block3_conv3/Relu:0']

In [ ]:
for tensor_name in tensor_name_list:
    display_patterns(tensor_name)